In [1]:
from sentence_transformers import SentenceTransformer, util
import torch

In [2]:
import os
import codecs
import re
import time
import numpy as np
import string

## Loading the model
Load the paraphrase-xlm-r-multilingual-v1 model, and use gpu if it is available

In [3]:
model_cpu = SentenceTransformer('paraphrase-xlm-r-multilingual-v1')
model = SentenceTransformer('paraphrase-xlm-r-multilingual-v1')
if torch.cuda.is_available():
    model = model.to(torch.device("cuda"))
print(model.device)
print(model_cpu.device)

cuda:0
cpu


## Example
Test multi lingual sentence embeddings by doing cosine similarity between similar sentences in different languages

In [4]:
# Two lists of sentences
sentences1 = ['The cat sits outside',
             'A man is playing guitar',
             'The new movie is awesome']

sentences2 = ['Мачка седи напољу',
              'Човек свира гитару',
              'Нови филм је сјајан']

#Compute embedding for both lists
embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)

#Compute cosine-similarits
cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)

#Output the pairs with their score
for i in range(len(sentences1)):
    print("{} \t\t {} \t\t Score: {:.4f}".format(sentences1[i], sentences2[i], cosine_scores[i][i]))


The cat sits outside 		 Мачка седи напољу 		 Score: 0.9889
A man is playing guitar 		 Човек свира гитару 		 Score: 0.9719
The new movie is awesome 		 Нови филм је сјајан 		 Score: 0.8863


## Embed the documents

### Load the dataset

In [5]:
stopwords = ["баш","без","биће","био","бити","близу","број","дана","данас","доћи","добар","добити","док","доле","дошао","други","дуж","два","често","чији","где","горе","хвала","и","ићи","иако","иде","има","имам","имао","испод","између","изнад","изван","изволи","један","једини","једном","јесте","још","јуче","кад","како","као","кога","која","које","који","кроз","мали","мањи","мисли","много","моћи","могу","мора","морао","на","наћи","наш","негде","него","некад","неки","немам","нешто","није","ниједан","никада","нисмо","ништа","њега","његов","њен","њих","њихов","око","около","она","онај","они","оно","осим","остали","отишао","овако","овамо","овде","ове","ово","о","питати","почетак","поједини","после","поводом","правити","пре","преко","према","први","пут","радије","сада","смети","шта","ствар","стварно","сутра","сваки","све","свим","свугде","тачно","тада","тај","такође","тамо","тим", "у", "учинио","учинити","умало","унутра","употребити","узети","ваш","већина","веома","видео","више","захвалити","зашто","због","желео","жели","знати"]

In [6]:
def hasNumbers(inputString):
     return any(char.isdigit() for char in inputString)

In [225]:
clan_re = re.compile("<p>[ \t\n]*Члан[ \t\n]*[0-9]*\..*?<\/p>")
clan_re.split("sadsadasd<p>Члан  13.  </p>asdj<p>Члан  12.  </p>sadsad")

['sadsadasd', 'asdj', 'sadsad']

In [306]:
start = time.time()
dataset_path = '../acts'
dataset = {}
TAG_RE = re.compile(r'<[^>]+>')
corpus_clans = []
for file_path in os.listdir(dataset_path):
    with codecs.open(os.path.join(dataset_path, file_path), 'r', encoding='utf-8') as f:
        data = f.read()
        clans = clan_re.split(data)
        clans_no_html = [TAG_RE.sub('', clan) for clan in clans] #remove html tags
        clans_no_white_spaces = [' '.join(clan.split()) for clan in clans_no_html]
        corpus_clans.append(clans_no_white_spaces)
        clans_no_punc = (clan.translate(str.maketrans('', '', string.punctuation + "„”–vi")) for clan in clans_no_white_spaces) #remove punctuation
        clans_lower = [clan.lower() for clan in clans_no_punc] #lowercase
        clans_no_stopwords = []
        for i, clan in enumerate(clans_lower):
            new_clan = clan.split()
            new_clan = [word for word in new_clan if word not in stopwords and not hasNumbers(word)]
            new_clan = ' '.join(new_clan)
            clans_no_stopwords.append(new_clan)
#         data = [row_clean for row in data if (row_clean := " ".join(row.split())) != ''] #remove extra whitespaces
        clans_clean = [clan_clean for clan in clans_no_stopwords if (clan_clean := " ".join(clan.split())) != ''] #remove extra whitespaces
        clans_cut = [clan[:500000] for clan in clans_clean]
        dataset[file_path] = clans_cut
end = time.time()
corpus = list(dataset.values())
print(end-start)

27.828669786453247


In [12]:
for key, value in dataset.items():
    if key == '1.html':
        for i in value:
            print(i)
            print('---------')
        break

﻿ закон изгледу употреби грба заставе химне републике србије “службени гласник рс од маја основне одредбе предмет закона
---------
овим законом уређују се изглед употреба грба заставе химне републике србије представљање републике србије
---------
грбом заставом химном републике србије представља се република србија изражава припадност републици србији грб застава химна републике србије се употребљавати само облику са садржином су утврђени уставом републике србије овим законом основно правило истицању грба заставе републике србије
---------
ако се грб односно застава републике србије истиче републици србији заједно са другим домаћим или страним грбовима или заставама грб односно застава републике србије ставља се почасно место ако овим законом друкчије одређено ограничења приликом употребе грба заставе републике србије
---------
заставу грб републике србије не може се исписати односно уписати нити се мењати изузетно ако је то посебним законом прописано застава или грб републике србије м

In [13]:
print(type(dataset['1.html']))

<class 'list'>


In [12]:
# model_cpu.encode(corpus[50], convert_to_tensor=True, show_progress_bar=True)

In [15]:
zakon_lens = [sum([len(clan) for clan in zakon]) for zakon in corpus]
sorted(enumerate(zakon_lens), key = lambda x: -x[1])

[(172, 549337),
 (1091, 506913),
 (1103, 505223),
 (50, 503546),
 (874, 452441),
 (661, 443683),
 (1090, 425514),
 (426, 397067),
 (474, 393465),
 (885, 369323),
 (331, 352447),
 (374, 348669),
 (869, 329602),
 (558, 326736),
 (193, 325690),
 (229, 318095),
 (447, 311478),
 (847, 310791),
 (389, 299516),
 (211, 297631),
 (363, 297492),
 (800, 295949),
 (247, 293503),
 (215, 286653),
 (844, 270204),
 (1120, 264727),
 (824, 263795),
 (901, 259029),
 (1053, 257008),
 (779, 253173),
 (510, 253021),
 (116, 252732),
 (602, 250452),
 (291, 232734),
 (804, 232390),
 (213, 230830),
 (617, 230149),
 (131, 225912),
 (656, 224568),
 (260, 220101),
 (593, 217760),
 (679, 217099),
 (486, 213568),
 (418, 212850),
 (160, 210126),
 (1058, 208574),
 (1065, 207355),
 (241, 206523),
 (722, 205471),
 (1043, 198410),
 (508, 191954),
 (421, 189958),
 (289, 187854),
 (663, 186534),
 (488, 184624),
 (189, 183397),
 (471, 175259),
 (695, 173018),
 (351, 171158),
 (74, 170519),
 (978, 169674),
 (768, 167659),
 (

### Embed the corpus

In [308]:
from tqdm import tqdm
embeddings = []
# corpus2 = corpus[:50] + corpus[51:]
# zakon_embedding = model.encode(corpus, convert_to_tensor=True, show_progress_bar=True)
for counter, zakon in enumerate(tqdm(corpus)):
#     print(f"{counter} len: {len(zakon)}, type: {type(zakon)}")
    
    zakon_embedding = model.encode(zakon, convert_to_tensor=True)
    embeddings.append(zakon_embedding)
    #time.sleep(0.01)

# embeddings = model.encode(corpus, convert_to_tensor=True, show_progress_bar=True)

100%|██████████| 1130/1130 [05:45<00:00,  3.27it/s]


In [33]:
print(type(embeddings))
print(type(embeddings[0]))
print(embeddings[2].shape)

<class 'list'>
<class 'torch.Tensor'>
torch.Size([45, 768])


In [317]:
def calculate_similarity(X, model, query, top_documents=10, top_sentences=3):
    """ Embeds the `query` via `model` and calculates the cosine similarity of
    the `query` and `X` (all the documents) and returns the `top_k` similar documents."""

    # Vectorize the query to the same length as documents
    query_vec = model.encode(query, convert_to_tensor=True)
    # Compute the cosine similarity between query_vec and all the documents
    scores = np.array([])
    top_sentence_indices = []
    for i,corpus_embedding in enumerate(embeddings):   
        cosine_scores = util.pytorch_cos_sim(query_vec, corpus_embedding)
        scores = np.append(scores, torch.sort(cosine_scores).values[0].numpy()[-1:-top_sentences-1:-1].mean())
        top_sentence_indices.append(torch.sort(cosine_scores).indices[0].numpy()[-1:-top_sentences-1:-1])
    # Sort the similar documents from the most similar to less similar and return the indices
    most_similar_doc_indices = np.argsort(scores, axis=0)[:-top_documents-1:-1].astype(int)
    top_sentence_indices = np.array(top_sentence_indices)
    return (most_similar_doc_indices, scores, top_sentence_indices[most_similar_doc_indices])

In [323]:
def show_similar_documents(df, cosine_similarities, similar_doc_indices, corpus, similar_doc_sentence_indices):
    """ Prints the most similar documents using indices in the `similar_doc_indices` vector."""
    counter = 1
    
    for i, index in enumerate(similar_doc_indices):
        print('Top-{}, Similarity = {}'.format(counter, cosine_similarities[index]))
        print('body: {}, '.format(df[index]))
        print('Most similar articles: ')
        for jandex in similar_doc_sentence_indices[i]:
            print(f'\t{corpus[index][jandex]}')
        print()
        counter += 1

In [319]:
corpus[0][1]

'потврђује се споразум владе републике србије владе републике молдавије сарадњи области одбране је потписан београду децембра године оригиналу српском молдавском енглеском језику'

In [334]:
top_documents = 10
top_sentences = 3
user_question = [
    'безбедност на раду'
]
search_start = time.time()
sim_vecs, cosine_similarities, top_sentence_indices = calculate_similarity(embeddings, 
                                                                           model, 
                                                                           user_question, 
                                                                           top_documents=top_documents,
                                                                           top_sentences=top_sentences)
search_time = time.time() - search_start
print("search time: {:.2f} ms".format(search_time * 1000))
print()
show_similar_documents(list(dataset.keys()), cosine_similarities, sim_vecs, corpus_clans, top_sentence_indices)

search time: 270.58 ms

Top-1, Similarity = 0.6523435711860657
body: 432.html, 
Most similar articles: 
	Запослени је дужан да примењује прописане мере за безбедан и здрав рад, да наменски користи средства за рад и опасне материје, да користи прописана средстава и опрему за личну заштиту на раду и да са њима пажљиво рукује, да не би угрозио своју безбедност и здравље као и безбедност и здравље других лица. Запослени је дужан да пре почетка рада прегледа своје радно место укључујући и средства за рад која користи, као и средства и опрему за личну заштиту на раду, и да у случају уочених недостатака извести послодавца или друго овлашћено лице. Запосленом је забрањено да самовољно искључује, мења или уклања безбедносне уређаје на средствима за рад. Пре напуштања радног места запослени је дужан да радно место и средства за рад остави у стању да не угрожавају друге запослене. *Службени гласник РС, број 91/2015
	Послодавац је дужан да изврши оспособљавање запосленог за безбедан и здрав рад ко

<ipython-input-317-57d180495117>:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  top_sentence_indices = np.array(top_sentence_indices)
